In [1]:
import numpy as np
import pandas as pd
from scipy import sparse

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, GRU
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
read_path = "/content/drive/My Drive/cs182final/yelp_review_training_dataset.jsonl"
data = pd.read_json(read_path, lines = True)

In [0]:
data = data.drop(["review_id"], axis =1)

In [5]:
data.head()

,text,stars
0,Total bill for this horrible service? Over $8G...,1
1,I *adore* Travis at the Hard Rock's new Kelly ...,5
2,I have to say that this office really has it t...,5
3,Went in for a lunch. Steak sandwich was delici...,5
4,Today was my second out of three sessions I ha...,1


In [6]:
data["text"] = data['text'].str.replace('[^\w\s]','').str.lower()
data["text"] = data['text'].str.replace('\d+', '')
data.head()

,text,stars
0,total bill for this horrible service over gs t...,1
1,i adore travis at the hard rocks new kelly car...,5
2,i have to say that this office really has it t...,5
3,went in for a lunch steak sandwich was delicio...,5
4,today was my second out of three sessions i ha...,1


In [7]:
data.shape

(533581, 2)

In [0]:
shuffled = data.sample(frac=1)
shuffled_sample = data.iloc[:10000]
train, validation = train_test_split(shuffled_sample, test_size=0.2)

In [9]:
train.shape, validation.shape

((8000, 2), (2000, 2))

In [10]:
train.head()

,text,stars
3463,i love this place elizabeth is the most profes...,5
4786,we stayed at the mgm grand to take a tour of t...,5
7818,pros\ngated community\nclean area\ngiant full ...,3
9387,extrêmement déçu par la qualité du service si ...,1
6824,actually \n\ni popped in here last night for a...,3


In [11]:
train = pd.get_dummies(train, columns = ['stars'])
train.head()

,text,stars_1,stars_2,stars_3,stars_4,stars_5
3463,i love this place elizabeth is the most profes...,0,0,0,0,1
4786,we stayed at the mgm grand to take a tour of t...,0,0,0,0,1
7818,pros\ngated community\nclean area\ngiant full ...,0,0,1,0,0
9387,extrêmement déçu par la qualité du service si ...,1,0,0,0,0
6824,actually \n\ni popped in here last night for a...,0,0,1,0,0


In [12]:
validation = pd.get_dummies(validation, columns = ['stars'])
train.shape, validation.shape

((8000, 6), (2000, 6))

In [13]:
train.head()

,text,stars_1,stars_2,stars_3,stars_4,stars_5
3463,i love this place elizabeth is the most profes...,0,0,0,0,1
4786,we stayed at the mgm grand to take a tour of t...,0,0,0,0,1
7818,pros\ngated community\nclean area\ngiant full ...,0,0,1,0,0
9387,extrêmement déçu par la qualité du service si ...,1,0,0,0,0
6824,actually \n\ni popped in here last night for a...,0,0,1,0,0


In [14]:
class_names = ['stars_1', 'stars_2', 'stars_3', 'stars_4', 'stars_5']
# Splitting off y variable
y = train[class_names].values
y

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [0]:
# max number of unique words 
max_features = 20000
# max number of words from review to use
maxlen = 200
# batch size
batchsize = 32

In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train['text'].values))

In [17]:
X_train = tokenizer.texts_to_sequences(train['text'].values)
X_validation = tokenizer.texts_to_sequences(validation['text'].values)
x_train = pad_sequences(X_train, maxlen = maxlen)
x_validation = pad_sequences(X_validation, maxlen = maxlen)
x_train.shape

(8000, 200)

In [18]:
model = Sequential()
model.add(Embedding(max_features, 200, input_length=maxlen))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(x_train, y, batch_size = batchsize, epochs = 20)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
8000/8000 [==============================] - 80s 10ms/step - loss: 1.0885 - accuracy: 0.5749
Epoch 2/20
8000/8000 [==============================] - 80s 10ms/step - loss: 0.7197 - accuracy: 0.7151
Epoch 3/20
8000/8000 [==============================] - 79s 10ms/step - loss: 0.5651 - accuracy: 0.7816
Epoch 4/20
8000/8000 [==============================] - 85s 11ms/step - loss: 0.3737 - accuracy: 0.8639
Epoch 5/20
8000/8000 [==============================] - 80s 10ms/step - loss: 0.2572 - accuracy: 0.9155
Epoch 6/20
8000/8000 [==============================] - 79s 10ms/step - loss: 0.1552 - accuracy: 0.9511
Epoch 7/20
8000/8000 [==============================] - 79s 10ms/step - loss: 0.1382 - accuracy: 0.9574
Epoch 8/20
8000/8000 [==============================] - 79s 10ms/step - loss: 0.0839 - accuracy: 0.9758
Epoch 9/20
8000/8000 [==============================] - 79s 10ms/step - loss: 0.0535 - accuracy: 0.9851
Epoch 10/20
8000/8000 [==============================] - 79s 10m

In [25]:
y_validation = model.predict([x_validation], batch_size=batchsize, verbose = 1)

2000/2000 [==============================] - 4s 2ms/step


In [26]:
model.evaluate(x_validation, validation[class_names].values, verbose = 1, batch_size=batchsize)

2000/2000 [==============================] - 4s 2ms/step


[2.3191450176239012, 0.6140000224113464]

In [0]:
# v = metrics.classification_report(np.argmax(test[class_names].values, axis = 1),np.argmax(y_test, axis = 1))
# print(v)